In [1]:
%matplotlib inline
from constants import *
from os.path import join
import matplotlib.pyplot as plt
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook
from gensim.models import Word2Vec, Doc2Vec

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

tqdm_notebook.pandas()
pd.options.display.max_columns = 30

In [77]:
file = join(TPX_PATH, 'dewac_nbfiles01_topic-candidates.csv')
topics = pd.read_csv(file)
topics = (
    topics
    .query('metric == "ref" and param_id == "e42" and nb_topics == 100')
    .drop(topics.columns[:5], axis=1)
    .reset_index(drop=True)
    .iloc[:20, :]
)
topics

,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9
0,Partei,PDS,Arbeiter,Kampf,Kommunist,DDR,Sozialismus,Gewerkschaft,Genosse,Kraft
1,König,Frau,Mann,Mädchen,Schiff,Hand,Wasser,Märchen,Tag,Vater
2,Lebenspartnerschaft,Ehe,Lebenspartner,Schwuler,Partner,Lesbe,Ehegatte,Abs.,Paar,Partnerschaft
3,BetrVG,Betriebsrat,Abs.,Nr.,Betrieb,OLG,Arbeitgeber,ABR,Mitbestimmungsrecht,Arbeitgeberin
4,Gesellschaft,Aufsichtsrat,Unternehmen,Vorstand,Aktionär,Aktie,Hauptversammlung,AktG,Kodex,Vertrauen
5,Artikel,Gesetz,Art,Mitglied,König,Verfassung,Recht,Präsident,Staat,Regierung
6,Kirche,Beamter,Bischof,Papst,Zeit,Kardinal,katholisch_Kirche,Rom,Priester,Erzbischof
7,Entscheidung,Antrag,Verfahren,Gericht,Fall,Grund,Satz,Antragsteller,Behörde,Anordnung
8,Richter,Staatsanwaltschaft,Fall,Aussage,Gericht,Beweis,Dr.,Impfung,Herr,Prozess
9,Zivildienst,Wehrpflicht,Bundeswehr,Wehrpflichtiger,Monat,Dienst,Kriegsdienstverweigerer,Soldat,Zivildienstleistende,Einberufung


In [43]:
file = join(TPX_PATH, 'dewac_nbfiles01_label-candidates_full.csv')
full = pd.read_csv(file)
full = full.rename(columns={'Unnamed: 6': 'label_method'})
#print(full.head())
simple = (
    full
    .query('metric == "ref" and param_id == "e42" and nb_topics == 100 and label_method == "comb"')
    .drop(full.columns[:7], axis=1)
    .reset_index(drop=True)
    .applymap(lambda x: eval(x)[0])
    .iloc[:20, :]
)
simple

,label0,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,label11,label12,label13,label14,label15,label16,label17,label18,label19
0,kommunistisch_Partei,kommunistisch_Partei_Deutschland,Arbeiterpartei,sozialistisch_Partei,bürgerlich_Partei,Sozialdemokratie,Massenorganisation,frei_Gewerkschaft,demokratisch_Sozialismus,Socjaldemokracja_Rzeczypospolitej_Polskiej,sozialdemokratisch_Partei_Deutschland,sozialdemokratisch_Partei,Bolschewik,rot_Ruhrarmee,Kommunismus,Einheitsfront,Aktion_demokratisch_Fortschritt,sozialdemokratisch_Arbeiterpartei,gothaer_Programm,unabhängig_Gewerkschaftsopposition
1,Pittheu,Kretheu,Jüngling,Hreidmar,Argeia,Eriphyle,Iole,Euadne,Deiphontes,Deiphobo,Johann_von_England,nach_Haus,Knabe,Tochter_der_König,Ninigi,Chrysippos,Magd,Der_Vater,Stiefmutter,Anna
2,gleichgeschlechtlich_Ehe,Lebenspartnerschaftsgesetz,Verpartnerung,gleichgeschlechtlich,Lebensgemeinschaft,lesbisch,Scheidung,Eherecht,Gütertrennung,Homosexualität,Ehescheidung,Regenbogenfamilie,Lebensformenpolitik,Abstandsgebot,sexuell_Identität,Partnerschaftsgesetz,Eingetragene_Partnerschaft,Sexualpartner,Ehepakt,elterlich_Sorge
3,Personalvertretungsgesetz,Betriebsrat,Beschäftigte,Bundespersonalvertretungsgesetz,Gesamtbetriebsrat,Arbeitnehmer,Betriebsvereinbarung,Betriebsbuße,Drittelbeteiligungsgesetz,Weiterbeschäftigungsanspruch,Arbeitsrecht,Tarifvertragspartei,Tarifvertrag,Arbeitsverhältnis,Betriebsrentengesetz,Betriebsänderung,Belegschaft,Beschlussverfahren,Verbrauchervertrag,Kündigungsschutzgesetz
4,Aktionär,Vorstandsvorsitzende,Aktiengesellschaft,Anteilseigner,Grundkapital,Geschäftsanteil,Kapitalerhöhung,Gesellschafter,Stammaktie,One_Equity_Partner,Board_of_Director,Aufsichtsrat,Gesellschafterversammlung,Generalversammlung,Hauptversammlung,Altaktionär,Aktienrecht,operativ_Geschäft,Kommanditgesellschaft_auf_Aktie,Vorzugsaktie
5,Regierungschef,Legislative,Verfassung,Verfassungsrecht,Volksvertretung,Staatsrat,amerikanisch_Menschenrechtskonvention,Vorsitzende,Gesetzgebung,Judikative,Staatsoberhaupt,Rechtsgrundlage,Statut,Abkomme,Exekutive,Mitgliedstaat,Dekret,Verfassungsänderung,allgemein_Gewaltverbot,Senat
6,päpstlich_Legat,Weihbischof,Kurienbischof,Erzbistum_Turin,Suffraganbischof,Erzbischof,Erzpriester,Diözesanbischof,Bischof,apostolisch_Delegat,Metropolit,Dompropst,Pontifikat,Papst,Bischofssitz,Offizial,Diözese,Generalvikar,Kathedrale,James_Robert_Knox
7,Kläger,Prozessgericht,Hauptverfahren,Prozessurteil,Angeklagte,Zulässigkeit,Vollstreckbarkeit,Teilurteil,sofortig_Vollziehung,Ablehnungsbescheid,Gerichtsverfahren,Hauptsacheverfahren,sofortig_Beschwerde,Anhörung,Zwischenurteil,Haftbeschwerde,Endurteil,Unzulässigkeit,Sprungrevision,maßgeblich_Zeitpunkt
8,Angeklagte,Kläger,Hauptverfahren,Gerichtsverfahren,Verdächtige,Strafsache,Anschuldigung,Tatverdacht,Strafkammer,Beschuldigter,Indiz,Freispruch,Prozessgericht,Beweismittel,Generalstaatsanwaltschaft,Staatsschutzkammer,Ermittlungsverfahren,Zwischenurteil,Gerichtsverhandlung,Staatsanwalt
9,weiß_Jahrgang,Ersatzreserve,Wehrpflicht,Berufssoldat,Einberufung,Soldat_auf_Zeit,freiwillig_Wehrdienst,Soldatengesetz,Zivildienst,Wehrsold,Wehrersatzdienst,Zivildienstgesetz,Rekrut,Soldatenversorgungsgesetz,Reserveoffizier,Gebirgsjäger,Dreijährig-Freiwilliger,Totalverweigerung,Wehrpflicht_in_Deutschland,Wehrsoldgesetz


In [45]:
from train_utils import EpochLogger, EpochSaver

d2vpath = join(EMB_PATH, 'd2v', 'd2v')
w2vpath = join(EMB_PATH, 'w2v', 'w2v')
print("Doc2Vec loading", d2vpath)
d2v = Doc2Vec.load(d2vpath)

print("Word2Vec loading", w2vpath)
w2v = Word2Vec.load(w2vpath)
print("models loaded")

Doc2Vec loading ../data/preprocessed/embeddings/d2v/d2v
Word2Vec loading ../data/preprocessed/embeddings/w2v/w2v
models loaded


In [46]:
d2v_wv = d2v.wv
d2v_dv = d2v.docvecs
w2v_wv = w2v.wv

d2vmodel.infer_vector(simple) #infer a d2v representation for a topic

del d2v
del w2v

In [55]:
w1 = simple.iloc[0,0]
vec1 = d2v_wv[w1]
w2 = simple.iloc[0,1]
vec2 = d2v_wv[w2]

In [60]:
# w2v_wv.cosine_similarities(vec, d2v_wv.vectors)
d2v_dv.distance(w1, w2), d2v_wv.distance(w1, w2), w2v_wv.distance(w1, w2)

(0.4414312243461609, 0.4072136878967285, 0.3993687629699707)

In [70]:
dists = d2v_wv.distances(w1, tuple(simple.iloc[0, 1:].values))
simple.iloc[0, 1:].to_frame().assign(d2v_wv_distances=dists).sort_values('d2v_wv_distances')

,0,d2v_wv_distances
label3,sozialistisch_Partei,0.246073
label14,Kommunismus,0.261926
label15,Einheitsfront,0.272340
label2,Arbeiterpartei,0.280178
label11,sozialdemokratisch_Partei,0.335489
label6,Massenorganisation,0.358646
label8,demokratisch_Sozialismus,0.396195
label5,Sozialdemokratie,0.402398
label12,Bolschewik,0.403119
label1,kommunistisch_Partei_Deutschland,0.407214


In [72]:
dists = w2v_wv.distances(w1, tuple(simple.iloc[0, 1:].values))
simple.iloc[0, 1:].to_frame().assign(w2v_wv_distances=dists).sort_values('w2v_wv_distances')

,0,w2v_wv_distances
label3,sozialistisch_Partei,0.294670
label2,Arbeiterpartei,0.295048
label11,sozialdemokratisch_Partei,0.347479
label14,Kommunismus,0.375449
label1,kommunistisch_Partei_Deutschland,0.399369
label5,Sozialdemokratie,0.399683
label15,Einheitsfront,0.401898
label6,Massenorganisation,0.462755
label8,demokratisch_Sozialismus,0.465640
label7,frei_Gewerkschaft,0.470516


In [81]:
has_vec = simple.iloc[0, :].map(lambda x: x in d2v_dv)
has_vec

label0     True
label1     True
label2     True
label3     True
label4     True
label5     True
label6     True
label7     True
label8     True
label9     True
label10    True
label11    True
label12    True
label13    True
label14    True
label15    True
label16    True
label17    True
label18    True
label19    True
Name: 0, dtype: bool

In [85]:
for word in simple.iloc[0, :]:
    print(word, d2v_dv[word][:3])

kommunistisch_Partei [0.14880304 0.2410298  0.00921005]
kommunistisch_Partei_Deutschland [ 0.01485822  0.23265952 -0.18486771]
Arbeiterpartei [ 0.39894158 -0.55674595  0.18212381]
sozialistisch_Partei [ 0.19551352 -0.08076364 -0.11990631]
bürgerlich_Partei [ 0.2925751  -0.00560244  0.06508418]
Sozialdemokratie [-0.00299884 -0.6104501   0.39840788]
Massenorganisation [-0.08753942  0.16435364  0.19077021]
frei_Gewerkschaft [ 0.11946975 -0.03126214 -0.21551421]
demokratisch_Sozialismus [ 0.35326654  0.45461226 -0.3754114 ]
Socjaldemokracja_Rzeczypospolitej_Polskiej [-0.05088069 -0.22011547 -0.05530345]
sozialdemokratisch_Partei_Deutschland [0.17208889 0.14877    0.11397935]
sozialdemokratisch_Partei [-0.08320348  0.22613658 -0.13806887]
Bolschewik [ 0.02478602 -0.10575379 -0.42004907]
rot_Ruhrarmee [ 0.36459985  0.15176307 -0.10058993]
Kommunismus [-0.11453343  0.4268518   0.15717927]
Einheitsfront [ 0.5541612  -0.16723068 -0.54578424]
Aktion_demokratisch_Fortschritt [-0.05979657 -0.38087

In [110]:
dists = d2v_dv.distances(w1, tuple(simple.iloc[0, 1:].values))
simple.iloc[0, 1:].to_frame().assign(d2v_dv_distances=dists).sort_values('d2v_dv_distances')

TypeError: not all arguments converted during string formatting

In [116]:
# ! nice !
labels = simple.iloc[0].copy()
for i in range(10):
    outlier = w2v_wv.doesnt_match(labels)
    print(outlier)
    labels = labels[labels != outlier]
print(labels)

rot_Ruhrarmee
gothaer_Programm
Aktion_demokratisch_Fortschritt
unabhängig_Gewerkschaftsopposition
Massenorganisation
Bolschewik
Socjaldemokracja_Rzeczypospolitej_Polskiej
Kommunismus
demokratisch_Sozialismus
Einheitsfront
label0                      kommunistisch_Partei
label1          kommunistisch_Partei_Deutschland
label2                            Arbeiterpartei
label3                      sozialistisch_Partei
label4                         bürgerlich_Partei
label5                          Sozialdemokratie
label7                         frei_Gewerkschaft
label10    sozialdemokratisch_Partei_Deutschland
label11                sozialdemokratisch_Partei
label17        sozialdemokratisch_Arbeiterpartei
Name: 0, dtype: object


In [118]:
topic = topics.iloc[0]
w2v_wv.most_similar(positive=topic, negative=None, topn=20, restrict_vocab=None, indexer=None)
#wv.most_similar([rand_vec], topn=5, indexer=annoy_index)

[('kommunistisch', 0.7275704145431519),
 ('kommunistisch_Partei', 0.7142001390457153),
 ('Sozialdemokrat', 0.7077106237411499),
 ('Arbeiterpartei', 0.707636833190918),
 ('KPD', 0.702242374420166),
 ('Sozialist', 0.701751708984375),
 ('sozialistisch', 0.7008939981460571),
 ('Sozialdemokratie', 0.695747971534729),
 ('Linkspartei', 0.6750216484069824),
 ('sozialdemokratisch_Partei', 0.6743425130844116),
 ('sozialdemokratisch', 0.6684965491294861),
 ('Arbeiterbewegung', 0.6586674451828003),
 ('sozialistisch_Partei', 0.6555250883102417),
 ('Parteimitglied', 0.6434710621833801),
 ('Gewerkschafter', 0.6433238983154297),
 ('Parteiführung', 0.6424713134765625),
 ('Volksfront', 0.6394513845443726),
 ('Die_Partei', 0.6379992961883545),
 ('Gewerkschaftsbewegung', 0.6360800266265869),
 ('frei_Gewerkschaft', 0.632095456123352)]

In [123]:
w2v_wv.most_similar_cosmul(positive=list(topic), negative=None, topn=20)

[('kommunistisch', 0.03645792976021767),
 ('kommunistisch_Partei', 0.0343996100127697),
 ('Arbeiterpartei', 0.03342046961188316),
 ('Sozialdemokrat', 0.033226363360881805),
 ('sozialistisch', 0.032760392874479294),
 ('KPD', 0.03267296403646469),
 ('Sozialist', 0.03214901313185692),
 ('Sozialdemokratie', 0.03196638822555542),
 ('sozialdemokratisch_Partei', 0.0290193110704422),
 ('Linkspartei', 0.028594257310032845),
 ('sozialdemokratisch', 0.028360987082123756),
 ('Arbeiterbewegung', 0.027008317410945892),
 ('sozialistisch_Partei', 0.02643883228302002),
 ('Parteiführung', 0.02553098089993),
 ('Volksfront', 0.025441884994506836),
 ('Gewerkschafter', 0.025384627282619476),
 ('Parteimitglied', 0.02523638866841793),
 ('Gewerkschaftsbewegung', 0.024693815037608147),
 ('frei_Gewerkschaft', 0.0242183618247509),
 ('Die_Partei', 0.024198325350880623)]

In [154]:
w2v_scores = topic.to_frame().apply(lambda x: pd.Series(w2v_wv.similar_by_word(x[0], topn=20, restrict_vocab=None)), axis=1).stack().apply(pd.Series).rename(columns={0:'term', 1:'score'})
w2v_scores.groupby('term').agg(['count', 'sum']).sort_values(('score', 'sum'), ascending=False)

score          
                              count       sum
term                                         
kommunistisch                     3  1.916268
Sozialdemokrat                    3  1.888201
KPD                               3  1.810673
Kommunismus                       2  1.433982
Linkspartei                       2  1.380561
sozialistisch                     2  1.345965
Arbeiterpartei                    2  1.327648
sozialistisch_Partei              2  1.305735
Arbeiterbewegung                  2  1.258019
Gewerkschafter                    2  1.228253
kommunistisch_Partei              2  1.228026
Parteimitglied                    2  1.213721
Kpd-Mitglied                      2  1.157464
Trotzkist                         2  1.142914
SED                               2  1.079617
deutsch_demokratisch_Republik     1  0.862029
Die_Partei                        1  0.820931
kämpfen                           1  0.774020
gewerkschaftlich                  1  0.768169
Wahlbündnis                       1  0.767385
Bundesrepublik                    1  0.766973
Die_Linke                         1  0.766315
BRD                               1  0.760405
politisch_Partei                  1  0.755516
Oppositionspartei                 1  0.755359
Ost-Berlin                        1  0.754305
Marxismus                         1  0.753668
Volkspartei                       1  0.739776
Der_Kampf                         1  0.737193
Sozialist                         1  0.733743
...                             ...       ...
Die_Kraft                         1  0.513868
Thälmann                          1  0.504023
Parteigenosse                     1  0.500983
rot_Hilfe                         1  0.491420
Komintern                         1  0.485952
Kamerad                           1  0.483451
Gesetz                            1  0.479620
Liebknecht                        1  0.476753
Parteifunktionär                  1  0.474569
wirksam                           1  0.474116
treibend                          1  0.466206
Revolutionär                      1  0.466027
Bewegung                          1  0.461314
Ernst_Thälmann                    1  0.457790
Parteiführung                     1  0.457220
Wirkung                           1  0.456929
Zk-Mitglied                       1  0.456518
anderer_Kraft                     1  0.432759
Gegenkraft                        1  0.418986
Macht                             1  0.418620
tretend                           1  0.418179
Inkrafttreten                     1  0.414207
Stärke                            1  0.412680
kraft                             1  0.406520
Widerstand                        1  0.406055
Das_Gesetz                        1  0.403749
Zwangskraft                       1  0.401773
Wirkt                             1  0.401426
Energie                           1  0.400949
Widerstandskraft                  1  0.400275

[182 rows x 2 columns]

In [156]:
d2v_scores = topic.to_frame().apply(lambda x: pd.Series(d2v_wv.similar_by_word(x[0], topn=20, restrict_vocab=None)), axis=1).stack().apply(pd.Series).rename(columns={0:'term', 1:'score'})
d2v_scores.groupby('term').agg(['count', 'sum']).sort_values(('score', 'sum'), ascending=False)

score          
                              count       sum
term                                         
Kommunismus                       2  1.574009
sozialistisch                     2  1.552963
kommunistisch                     2  1.500283
gewerkschaftlich                  2  1.488509
Sozialist                         2  1.485968
Arbeiterbewegung                  2  1.427791
Streik                            2  1.414265
demokratisch_Sozialismus          2  1.402648
Arbeitskampf                      2  1.362184
Trotzkist                         2  1.340160
Kpd-Mitglied                      2  1.305032
KPD                               2  1.304928
Kommunistin                       2  1.292288
kämpfen                           1  0.885046
deutsch_demokratisch_Republik     1  0.856135
Die_Linke                         1  0.853391
gewerkschaftliche                 1  0.845275
Marxismus                         1  0.839779
Die_Partei                        1  0.836632
kommunistisch_Partei              1  0.831114
Parteivorsitzender                1  0.828297
marxistisch                       1  0.821943
Gewerkschaftsmitglied             1  0.818353
kandidieren                       1  0.818348
Mandat                            1  0.817909
Parlament                         1  0.814652
Kapitalismus                      1  0.811166
Wahlbündnis                       1  0.810763
WASG                              1  0.806454
parteiintern                      1  0.803055
...                             ...       ...
lösen                             1  0.601118
denunzieren                       1  0.596055
entgegen                          1  0.595418
fordern                           1  0.594669
erklären                          1  0.589352
Widerstand                        1  0.587902
Bewegung                          1  0.580737
verhaften                         1  0.576548
Verhaftung                        1  0.573729
konterrevolutionär                1  0.573209
Parteiarbeit                      1  0.572995
indem                             1  0.572355
Verhaftungswelle                  1  0.571892
Zentralkomitee                    1  0.571363
Kpd-Funktionär                    1  0.570785
Zuchthausstrafe                   1  0.570764
Sympathisant                      1  0.569994
dadurch                           1  0.568401
Hochverrat                        1  0.567586
verhindern                        1  0.565758
gemäß                             1  0.563363
Wille                             1  0.559268
versuchen                         1  0.558938
geschehen                         1  0.556896
Aufgabe                           1  0.550651
ausüben                           1  0.549721
unabhängig                        1  0.547636
ändern                            1  0.545129
Weise                             1  0.542842
getreten                          1  0.542773

[187 rows x 2 columns]

In [162]:
# combined
comb_scores = w2v_scores.append(d2v_scores).groupby('term').agg(['count', 'sum']).sort_values(('score', 'sum'), ascending=False)
comb_scores

score          
                              count       sum
term                                         
kommunistisch                     5  3.416551
KPD                               5  3.115601
Kommunismus                       4  3.007992
sozialistisch                     4  2.898928
Arbeiterbewegung                  4  2.685810
Sozialdemokrat                    4  2.620496
Trotzkist                         4  2.483075
Kpd-Mitglied                      4  2.462496
gewerkschaftlich                  3  2.256677
Sozialist                         3  2.219711
Linkspartei                       3  2.145905
Streik                            3  2.064514
kommunistisch_Partei              3  2.059141
demokratisch_Sozialismus          3  2.018154
Gewerkschafter                    3  2.007068
Arbeitskampf                      3  1.968626
deutsch_demokratisch_Republik     2  1.718164
kämpfen                           2  1.659066
Die_Partei                        2  1.657563
Die_Linke                         2  1.619706
Marxismus                         2  1.593447
Wahlbündnis                       2  1.578149
gewerkschaftliche                 2  1.563624
politisch_Partei                  2  1.552476
Bundesrepublik                    2  1.542400
Parteivorsitzender                2  1.539072
Kapitalismus                      2  1.537759
WASG                              2  1.532479
Ost-Berlin                        2  1.531346
BRD                               2  1.525272
...                             ...       ...
Gegenangriff                      1  0.554092
Aufgabe                           1  0.550651
ausüben                           1  0.549721
unabhängig                        1  0.547636
ändern                            1  0.545129
Weise                             1  0.542842
Die_Kraft                         1  0.513868
Thälmann                          1  0.504023
Parteigenosse                     1  0.500983
Kamerad                           1  0.483451
Liebknecht                        1  0.476753
Parteifunktionär                  1  0.474569
wirksam                           1  0.474116
treibend                          1  0.466206
Revolutionär                      1  0.466027
Ernst_Thälmann                    1  0.457790
Wirkung                           1  0.456929
Zk-Mitglied                       1  0.456518
anderer_Kraft                     1  0.432759
Gegenkraft                        1  0.418986
Macht                             1  0.418620
tretend                           1  0.418179
Inkrafttreten                     1  0.414207
Stärke                            1  0.412680
kraft                             1  0.406520
Das_Gesetz                        1  0.403749
Zwangskraft                       1  0.401773
Wirkt                             1  0.401426
Energie                           1  0.400949
Widerstandskraft                  1  0.400275

[276 rows x 2 columns]

In [164]:
comb_scores[comb_scores.index.isin(phrases.token)]

score          
                                    count       sum
term                                               
Kommunismus                             4  3.007992
Arbeiterbewegung                        4  2.685810
Linkspartei                             3  2.145905
Streik                                  3  2.064514
kommunistisch_Partei                    3  2.059141
demokratisch_Sozialismus                3  2.018154
Arbeitskampf                            3  1.968626
deutsch_demokratisch_Republik           2  1.718164
Die_Partei                              2  1.657563
Die_Linke                               2  1.619706
Marxismus                               2  1.593447
Wahlbündnis                             2  1.578149
politisch_Partei                        2  1.552476
Parteivorsitzender                      2  1.539072
Kapitalismus                            2  1.537759
Ost-Berlin                              2  1.531346
BRD                                     2  1.525272
Der_Kampf                               2  1.504382
Bündnis_90/Die_Grüne                    2  1.492059
Partei_der_demokratisch_Sozialismus     2  1.475292
deutsch_Wiedervereinigung               2  1.469624
Klassenkampf                            2  1.420565
frei_Gewerkschaft                       2  1.417674
Ideologie                               2  1.396868
Sozialdemokratie                        2  1.395365
FDP                                     2  1.393800
Staatssozialismus                       2  1.390718
Westdeutschland                         2  1.385249
Diktatur_der_Proletariat                2  1.375059
Gewerkschaftsbund                       2  1.370322
...                                   ...       ...
IG_Metall                               1  0.590126
Arbeiterin                              1  0.586439
frei_Wähler                             1  0.585403
Arbeitnehmer                            1  0.580142
Belegschaft                             1  0.578942
Tagelöhner                              1  0.577690
Verhaftung                              1  0.573729
Zentralkomitee                          1  0.571363
Feldzug                                 1  0.570981
Sympathisant                            1  0.569994
Hochverrat                              1  0.567586
Facharbeiter                            1  0.564527
Auseinandersetzung                      1  0.562895
Scharmützel                             1  0.561986
Wille                                   1  0.559268
Straßenkampf                            1  0.558997
Aufgabe                                 1  0.550651
unabhängig                              1  0.547636
Weise                                   1  0.542842
Die_Kraft                               1  0.513868
Kamerad                                 1  0.483451
Liebknecht                              1  0.476753
Ernst_Thälmann                          1  0.457790
Wirkung                                 1  0.456929
anderer_Kraft                           1  0.432759
Macht                                   1  0.418620
Stärke                                  1  0.412680
Das_Gesetz                              1  0.403749
Zwangskraft                             1  0.401773
Energie                                 1  0.400949

[130 rows x 2 columns]

In [ ]:
w2v_wv.similarity(w1, w2)